In [ ]:
import pandas as pd
import numpy as np

In [ ]:
converters = {"Address": int}
registers = pd.read_csv("registers.csv", delimiter=",", dtype="string", converters=converters, keep_default_na=True)
registers.fillna("None", inplace=True)

other_cols     = registers.columns.difference(list(converters.keys()))
registers[other_cols] = registers[other_cols].applymap(lambda x: None if (x == "None") else x)



masks = pd.read_csv("masks.csv", delimiter=",")
print(registers)

print(masks)

In [ ]:
datatype_conversion = {
    "U8":"uint8_t",
    "S8":"int16_t",
    "U16":"uint16_t",
    "S16":"int16_t",
    "U32":"uint32_t",
    "S32":"int32_t",
    "U64":"uint64_t",
    "S64":"int64_t",
    "Float":"float",
}

In [ ]:
txt_fnc = lambda row: f"""	{datatype_conversion[row.Format]} {row.Identifier};
"""
registers_data = registers.apply(txt_fnc, axis=1).values.tolist()

header_register_s =\
"""//************************************************************************//
//* Registers' structure                                                 *//
//************************************************************************//
typedef struct
{
""" + f"{''.join(registers_data)}" + """
} AppRegs;
"""


In [ ]:
txt_fnc = lambda row: f"""#define {row.Identifier}	{row.Address} //\
 {row.Format}{f"[{row.ArrayLength}]" if row.ArrayLength is not None else ""}\
	{row.MaskFamily if pd.isnull(row.MaskFamily) is False else ""}\
			{row.Description if pd.isnull(row.Description) is False else ""}
"""


registers_data = registers.apply(txt_fnc, axis=1).values.tolist()

header_register =\
"""/************************************************************************/
/* Registers' address                                                   */
/************************************************************************/
/* Registers */
""" + f"{''.join(registers_data)}"




In [ ]:
txt_fnc = lambda row: f"""#define {row.Identifier}	{row.BitMask if "<<" not in row.BitMask else f"({row.BitMask})"}	// {row.Format}	{row.Description if pd.isnull(row.Description) is False else ""}
"""
mask_data = masks.apply(txt_fnc, axis=1).values.tolist()

header_mask =\
"""/************************************************************************/
/* Registers' bits                                                      */
/************************************************************************/
""" + f"{''.join(mask_data)}"

print(header_mask)


In [ ]:
f = open("header.h", "w")
f.writelines([header_register_s, "\n", header_register, "\n", header_mask])
f.close()
